In [1]:
import pickle

In [3]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
# for pickling the model
from tensorflow.keras.models import load_model, save_model
import pickle

# Assuming dcgan is your DCGAN model with a generator
generator_model = dcgan.generator

# Save the generator model
save_model(generator_model, 'generator_model.h5')

# Load the generator model
loaded_generator = load_model('generator_model.h5')




# Serialize the generator model
with open('generator_model.pkl', 'wb') as f:
    pickle.dump(generator_model, f)

# Deserialize the generator model
with open('generator_model.pkl', 'rb') as f:
    loaded_generator = pickle.load(f)


In [8]:
%%writefile app.py
# %%writefile app.py
import streamlit as st
import pickle
import tensorflow as tf
import numpy as np
from PIL import Image
from io import BytesIO

def generate_faces(generator, num_images, randomness_level):
    generated_images = []

    for _ in range(num_images):
        # Generate random noise
        random_noise = np.random.normal(size=(1, 100))

        # Adjust randomness based on user input
        random_noise *= randomness_level

        # Generate face using the loaded generator
        generated_image = generator.predict(random_noise)[0]

        # Denormalize the image
        generated_image = (generated_image + 1) * 127.5

        # Resize to 20x20
        generated_image = tf.image.resize(generated_image, (20, 20)).numpy()

        # Convert to PIL Image for display
        pil_image = Image.fromarray(generated_image.astype('uint8'))

        # Append the generated image to the list
        generated_images.append(pil_image)

    return generated_images

def main():
    st.title("Anime Face Generator")

    # Load the generator model
    with open('generator_model.pkl', 'rb') as f:
        loaded_generator = pickle.load(f)

    # Number of Images
    num_images = st.slider("Number of Images", 1, 5, 2)

    # Randomness Level
    randomness_level = st.slider("Randomness Level", 0.0, 1.0, 0.5, step=0.1)

    # Show noise as the starting point
    starting_point_noise_images = []
    for _ in range(num_images):
        starting_point_noise = np.random.normal(size=(1, 100))
        starting_point_noise_image = np.clip((starting_point_noise + 1) * 127.5, 0, 255).astype('uint8')
        starting_point_pil_image = Image.fromarray(starting_point_noise_image[0], mode='L')  # Convert to grayscale
        resized_noise_image = starting_point_pil_image.resize((20, 20))
        starting_point_noise_images.append(resized_noise_image)

    # Display the noise images
    st.write("This is where we started")
    columns = st.columns(num_images)
    for i, (img, col) in enumerate(zip(starting_point_noise_images, columns)):
        # Display image
        col.image(img, caption=f"Noise Image {i + 1}", width=20, use_column_width=True)



    # Generate Faces Button
    if st.button("Generate Faces"):
        generated_images = generate_faces(loaded_generator, num_images, randomness_level)

        # Title for generated images
        st.write("This is where our model takes us")

        # Calculate the number of rows and columns
        num_columns = min(num_images, 5)
        num_rows = (num_images + num_columns - 1) // num_columns

        # Display the generated images in rows and columns
        for row in range(num_rows):
            row_images = generated_images[row * num_columns: (row + 1) * num_columns]
            columns = st.columns(num_columns)

            for i, (img, col) in enumerate(zip(row_images, columns)):
                # Display image
                col.image(img, caption=f"Generated Anime Face {i + 1}", width=20, use_column_width=True)

                # Download button
                img_bytes = BytesIO()
                img.save(img_bytes, format='PNG')
                col.download_button(label=f"Download Image {i + 1}", data=img_bytes, file_name=f"generated_image_{i + 1}.png", mime='image/png')

if __name__ == '__main__':
    main()



Overwriting app.py


In [9]:
! wget -q -O - ipv4.icanhazip.com

35.245.96.75


In [10]:
! streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.96.75:8501

npx: installed 22 in 2.171s
your url is: https://dry-bananas-tap.loca.lt
2023-11-21 06:24:24.606046: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 06:24:24.606118: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 06:24:24.606166: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 06:24:24.619180: I tensorflow/core/platform/cpu_feature_gua